In [ ]:
# Import packages and set parameters

import numpy as np
import matplotlib.pyplot as plt

from multiprocessing import Pool, cpu_count

from tqdm import tqdm

import datetime as dt

import sklearn
from sklearn import ensemble, cluster

from .utils import *

import scipy.cluster.hierarchy as hc
from scipy.spatial.distance import pdist, squareform

TR = 0.720

In [ ]:
from os.path import join as OSjoin
import nibabel as nib

def importHCP(dpath, dname, flip=True):
    fname = OSjoin(dpath,dname)
    img = nib.load(fname)
    print('For filename: ' + str(fname) + '\n' + 'img shape is: ' + str(img.shape))
    D = img.get_fdata() #projection
    if flip:
        D = D.T+0.0
    print('Output shape is ' + str(D.shape))    
    return D

In [ ]:
# Import data

dpath = '/data/jbillings/'
dname = 'HCP_S1200_812_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr_recon2.dconn.nii'

DC = importHCP(dpath,dname)

dpath = '/data/jbillings/'
dname = 'HCP_S1200_812_rfMRI_MSMAll_groupPCA_d4500_Eigenmaps_recon2.dtseries.nii'

PC = importHCP(dpath,dname)

dpath = '/data/jbillings/glasser'
dname = 'HCP_S1200_812_rfMRI_MSMAll_groupPCA_d4500_Eigenmaps_recon2.dtseries.nii'

GL = importHCP(dpath,dname)

In [ ]:
# Cluster DC
zdc = hc.average(squareform(DC,force='tovector'))
nclusts = 30000
clusts = hc.fcluster(zdc,nclust,criterion='maxclust')

In [ ]:
# Rebuild DC as dc from DC-clusters
unc = np.unique(clusts)
nmc = unc.size
dc = np.zeros([nmc,nmc])
meandc = np.zeros([nmc,1])
pcacorr = np.zeros([nmc,1])
gljac = np.zeros([nmc,1])

inds = []
for i in range(0,nmc,1):    
    inds.append(clusts==unc[i])
    divisor = inds[i].size**2-inds[i].size
    meandc[i] = sum(DC[inds[i],inds[i]])/divisor
    pcacorr[i] = np.mean(pdist(PC[inds[i],:],metric='correlation'))
    gljac[i] = np.unique(GL[inds[i]]).size

for i in range(0,nmc-1,1):    
    for j in range(i+1,nmc,1):
        temp = 0
        for ii in inds[i]:
            for jj in inds[j]:
                temp += DC[ii,jj]
        dc[i,j] = temp/(inds[i].size*inds[j].size)

dc = dc + dc.T